# Working with Instrumental Descriptions

the instrumental description is loaded by the event source, and consists of a hierarchy of classes in the ctapipe.instrument module, the base of which is the `SubarrayDescription`

First, let's open a file and load a single event so we get the instrument info in the event.inst container.

In [ ]:
from ctapipe.utils.datasets import get_dataset_path
from ctapipe.io import event_source
import numpy as np

#filename = get_dataset_path("gamma_test_large.simtel.gz") # try this one as well
filename = get_dataset_path("gamma_test_large.simtel.gz")  

# just get the first event in the file:
with event_source(filename, max_events=1) as source:
    event = next(iter(source))


## the SubarrayDescription:

In [ ]:
subarray = event.inst.subarray

subarray.info()

In [ ]:
subarray.to_table()

You can also get a table of just the `OpticsDescriptions` (`CameraGeometry` is more complex and can't be stored on a single table row, so each one can be converted to a table separately)

In [ ]:
subarray.to_table(kind='optics')

Make a sub-array with only SC-type telescopes:

In [ ]:
tab = subarray.to_table()
sc_tels = tab[tab['num_mirrors'] == 2]['tel_id']  # select tel_id of entries where the mirror type is SC
newsub = subarray.select_subarray("SCTels", sc_tels)
newsub.info()

can also do this by using `Table.group_by`

In [ ]:
gtab = tab.group_by('num_mirrors')
sc = gtab.groups[0]
newsub = subarray.select_subarray("SCTels", sc['tel_id'])
newsub.info()

## Explore some of the details of the telescopes

In [ ]:
tel = subarray.tel[5]
tel

In [ ]:
tel.optics.mirror_area

In [ ]:
tel.optics.num_mirror_tiles

In [ ]:
tel.optics.equivalent_focal_length

In [ ]:
tel.camera

In [ ]:
tel.camera.pix_x

In [ ]:
%matplotlib inline
from ctapipe.visualization import CameraDisplay
CameraDisplay(tel.camera)

In [ ]:
CameraDisplay(subarray.tel[98].camera)

## Plot the subarray

We'll make a subarray by telescope type and plot each separately, so they appear in different colors.  We also calculate the radius using the mirror area (and exagerate it a bit).

This is just for debugging and info, for any "real" use, a `visualization.ArrayDisplay` should be used

In [ ]:
subarray.peek()

In [ ]:
subarray.footprint

## Get info about the subarray in general

In [ ]:
subarray.telescope_types

In [ ]:
subarray.camera_types

In [ ]:
subarray.optics_types

In [ ]:
from astropy.coordinates import SkyCoord
from ctapipe.coordinates import GroundFrame
center = SkyCoord("10.0 m", "2.0 m", "0.0 m", frame='groundframe')
coords = subarray.tel_coords  # a flat list of coordinates by tel_index
coords.separation(center)

## Telescope IDs vs Indices

Note that `subarray.tel` is a dict mapped by `tel_id` (the indentifying number of a telescope).  It is  possible to have telescope IDs that do not start at 0, are not contiguouous (e.g. if a subarray is selected).  Some functions and properties like `tel_coords` are numpy arrays (not dicts) so they are not mapped to the telescope ID, but rather the *index* within this SubarrayDescription. To convert between the two concepts you can do:

In [ ]:
subarray.tel_ids_to_indices([1,5,23])

or you can get the indexing array directly in numpy or dict form:

In [ ]:
subarray.tel_index_array

In [ ]:
subarray.tel_index_array[[1,5,23]]

In [ ]:
subarray.tel_indices[1]  # this is a dict of tel_id -> tel_index, so we can only do one at once

In [ ]:
ids = subarray.get_tel_ids_for_type(subarray.telescope_types[0])
ids

In [ ]:
idx = subarray.tel_ids_to_indices(ids)
idx

In [ ]:
subarray.tel_coords[idx]

so, with that method you can quickly get many telescope positions at once (the alternative is to use the dict `positions` which maps `tel_id` to a position on the ground

In [ ]:
subarray.positions[1]

## Manipulate the subarray With Pandas
If you prefer working with *Pandas* `DataFrames` instead of *AstroPy* `Tables`, you can always convert between the two:

In [ ]:
tel_string = "LST_LST_LSTCam" # an LST with LST structure and LSTCam camera
df = subarray.to_table().to_pandas()
df.set_index('tel_id')

In [ ]:
g = df.groupby('tel_description')
g.groups

In [ ]:
g.groups[tel_string]

In [ ]:
df.loc[g.groups[tel_string]]

In [ ]:
lsts = subarray.select_subarray("LSTs", df.loc[g.groups[tel_string]]['tel_id'])
lsts.info()
lsts.peek()
lsts.footprint